In [ ]:
# policy_administration.py

import json
import os
import time
import logging
import psutil
import uuid
import requests
from flask import Flask, request, jsonify

app = Flask(__name__)

class PolicyAdministration:
    def __init__(self, policies_file='policies.json', meta_policies_file='meta_policies.json'):
        self.policies_file = policies_file
        self.meta_policies_file = meta_policies_file
        self.policies = {}
        self.meta_policies = []
        self.banned_ids = set()
        self.pep_urls = []  # List of PEP URLs
        self.entity_segments = {
            'iot_device1': 'production_network',
            'iot_device2': 'production_network',
            'user': 'user_network',
            'server': 'data_center',
            # Add more entities as needed
        }
        self.segment_sensitivity = {
            'production_network': 'high',
            'data_center': 'high',
            'user_network': 'low',
            'default_segment': 'low',
            # Add more segments as needed
        }
        self.pe_url = None  # URL of the Policy Engine
        self.load_policies()
        self.load_meta_policies()
        self.setup_logging()

    def setup_logging(self):
        # Configure the logging module
        logging.basicConfig(
            filename='policy_administration.log',
            level=logging.INFO,
            format='%(asctime)s %(levelname)s: %(message)s'
        )
        # Also log to console
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)

    def load_policies(self):
        if os.path.exists(self.policies_file):
            with open(self.policies_file, 'r') as f:
                self.policies = json.load(f)
            print(f"PA: Loaded policies from '{self.policies_file}'.")
            
            # Ensure 'state' is present in each policy
            for entity_id, policy in self.policies.items():
                if 'state' not in policy:
                    self.policies[entity_id]['state'] = 'Normal'
                    print(f"PA: Added default 'state'='Normal' to entity '{entity_id}'.")
        else:
            print(f"PA Error: Policies file '{self.policies_file}' not found. Using empty policies.")
            self.policies = {}

    def save_policies(self):
        try:
            with open(self.policies_file, 'w') as f:
                json.dump(self.policies, f, indent=4)
            print(f"PA: Saved policies to '{self.policies_file}'.")
        except Exception as e:
            print(f"PA Error: Error saving policies: {e}")

    def load_meta_policies(self):
        if os.path.exists(self.meta_policies_file):
            with open(self.meta_policies_file, 'r') as f:
                self.meta_policies = json.load(f)
            print(f"PA: Loaded meta-policies from '{self.meta_policies_file}'.")
        else:
            print(f"PA Error: Meta-policies file '{self.meta_policies_file}' not found. Using empty meta-policies.")
            self.meta_policies = []

    def register_pep(self, pep_url):
        self.pep_urls.append(pep_url)
        # Send current banned IDs to the PEP
        self.send_banned_ids_to_pep(pep_url)
        print(f"PA: PEP registered at '{pep_url}' and sent current banned IDs.")

    def send_banned_ids_to_pep(self, pep_url):
        """
        Sends the current banned IDs list to the PEP.
        """
        try:
            payload = {
                'banned_ids': list(self.banned_ids)
            }
            response = requests.post(f'{pep_url}/update_banned_ids', json=payload)
            response.raise_for_status()
            logging.info(f"PA: Sent banned IDs to PEP at '{pep_url}'.")
        except Exception as e:
            logging.error(f"PA Error: Failed to send banned IDs to PEP at '{pep_url}': {e}")

    def update_banned_ids(self):
        for pep_url in self.pep_urls:
            self.send_banned_ids_to_pep(pep_url)
        print(f"PA: Banned IDs list updated and sent to all PEPs.")

    def get_policy(self, entity_id):
        return self.policies.get(entity_id, {})

    def update_policy(self, entity_id, policy_data, request_id):
        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        if self.check_meta_policies(entity_id, policy_data):
            # Ensure 'state' is part of the policy_data
            if 'state' not in policy_data:
                policy_data['state'] = 'Normal'  # Default state
            self.policies[entity_id] = policy_data
            self.save_policies()
            print(f"PA: Policy updated for entity '{entity_id}'")
            self.notify_pe_policy_updated(entity_id, policy_data, request_id)
            update_status = 'success'
        else:
            print(f"PA Error: New policy for entity '{entity_id}' violates meta-policies. Policy not updated.")
            update_status = 'failure'

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data
        logging.info(
            f"Request ID: {request_id} | Entity ID: {entity_id} | Policy Update Status: {update_status} | "
            f"Processing Time: {processing_time:.6f}s | CPU Time: User={user_cpu_time:.6f}s System={system_cpu_time:.6f}s "
            f"Total={total_cpu_time:.6f}s"
        )

        return update_status

    def notify_pe_policy_updated(self, entity_id, policy_data, request_id):
        if self.pe_url:
            try:
                payload = {
                    'entity_id': entity_id,
                    'policy_data': policy_data,
                    'request_id': request_id
                }
                response = requests.post(f'{self.pe_url}/policy_updated', json=payload)
                response.raise_for_status()
                logging.info(f"PA: Notified PE of policy update for entity '{entity_id}'.")
            except Exception as e:
                logging.error(f"PA Error: Failed to notify PE of policy update: {e}")
        else:
            logging.warning(f"PA Warning: Policy Engine URL not set. Cannot notify PE of policy update.")

    def set_pe_url(self, pe_url):
        self.pe_url = pe_url

    def check_meta_policies(self, entity_id, policy_data):
        for meta_policy in self.meta_policies:
            if not self.evaluate_meta_policy(meta_policy, entity_id, policy_data):
                print(f"PA Error: Policy for entity '{entity_id}' violates meta-policy: {meta_policy['name']}")
                return False
        return True

    def evaluate_meta_policy(self, meta_policy, entity_id, policy_data):
        entity_type = policy_data.get('entity_type')
        if meta_policy.get('type') not in ['any', entity_type]:
            return True  # Meta-policy does not apply to this entity type

        condition = meta_policy.get('condition')
        try:
            # Prepare the local variables for eval
            local_vars = policy_data.copy()
            local_vars['includes'] = lambda a, b: all(item in a for item in b)
            local_vars['entity_id'] = entity_id
            # Add any other necessary variables to local_vars here
            logging.info(f"Evaluating meta-policy '{meta_policy['name']}' with variables: {local_vars}")
            # Evaluate the condition
            result = eval(condition, {}, local_vars)
            return result
        except Exception as e:
            print(f"PA Error: Failed to evaluate meta-policy '{meta_policy['name']}' for entity '{entity_id}': {e} and Meta policy conditions: {condition}")
            logging.error(f"PA Error: Failed to evaluate meta-policy '{meta_policy['name']}' for entity '{entity_id}': {e}")
            return False

    def receive_state_update(self, entity_id, new_state, request_id):
        """
        Receives state updates from the PE and adjusts policies accordingly.
        """
        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        policy = self.get_policy(entity_id)
        if not policy:
            policy = {
                'access_level': 'no-access',
                'state': new_state,
                # Add other default policies as needed
            }

        # Update the state in policy_data
        policy['state'] = new_state

        if new_state == 'Normal':
            # Reset access_level if necessary
            pass
        elif new_state == 'Alert':
            # Check the segment of the entity
            segment = self.entity_segments.get(entity_id, 'default_segment')
            sensitivity = self.segment_sensitivity.get(segment, 'low')  # Assuming 'high' or 'low'
            if sensitivity == 'high':
                # Change access policy to read-only
                policy['access_level'] = 'read-only'
            else:
                # Maybe less restrictive
                pass
        elif new_state == 'High Risk':
            # Change access policy to read-only
            policy['access_level'] = 'read-only'
        elif new_state == 'Quarantined':
            # Add entity to banned list and revoke all access
            self.banned_ids.add(entity_id)
            self.update_banned_ids()  # Notify PEPs
            policy['access_level'] = 'no-access'
        else:
            # Handle other states if needed
            pass

        # Update the policy
        update_status = self.update_policy(entity_id, policy, request_id)

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data
        logging.info(
            f"Request ID: {request_id} | Entity ID: {entity_id} | State Update: {new_state} | "
            f"Processing Time: {processing_time:.6f}s | CPU Time: User={user_cpu_time:.6f}s "
            f"System={system_cpu_time:.6f}s Total={total_cpu_time:.6f}s | Update Status: {update_status}"
        )

# Flask API Endpoints

@app.route('/register_pep', methods=['POST'])
def register_pep_endpoint():
    """
    API endpoint for PEPs to register with the PA.
    Expects JSON data with 'pep_url' and optional 'request_id'.
    """
    try:
        json_data = request.get_json()
        if not json_data:
            return jsonify({'error': 'No JSON data provided'}), 400

        pep_url = json_data.get('pep_url')
        request_id = json_data.get('request_id', str(uuid.uuid4()))

        if not pep_url:
            return jsonify({'error': 'Missing pep_url'}), 400

        PA_instance.register_pep(pep_url)

        response = {
            'request_id': request_id,
            'status': 'PEP registered successfully'
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in register_pep_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/get_policy', methods=['GET'])
def get_policy_endpoint():
    """
    API endpoint to get the policy for an entity.
    Expects query parameters 'entity_id' and optional 'request_id'.
    """
    try:
        entity_id = request.args.get('entity_id')
        request_id = request.args.get('request_id', str(uuid.uuid4()))

        if not entity_id:
            return jsonify({'error': 'Missing entity_id'}), 400

        policy = PA_instance.get_policy(entity_id)

        # Log the performance data (assuming minimal processing time)
        logging.info(f"Request ID: {request_id} | Entity ID: {entity_id} | Action: get_policy")

        response = {
            'request_id': request_id,
            'entity_id': entity_id,
            'policy': policy
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in get_policy_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/update_policy', methods=['POST'])
def update_policy_endpoint():
    """
    API endpoint to update the policy for an entity.
    Expects JSON data with 'entity_id', 'policy_data', and optional 'request_id'.
    """
    try:
        json_data = request.get_json()
        if not json_data:
            return jsonify({'error': 'No JSON data provided'}), 400

        entity_id = json_data.get('entity_id')
        policy_data = json_data.get('policy_data', {})
        request_id = json_data.get('request_id', str(uuid.uuid4()))

        if not entity_id:
            return jsonify({'error': 'Missing entity_id'}), 400
        if not policy_data:
            return jsonify({'error': 'Missing policy_data'}), 400

        update_status = PA_instance.update_policy(entity_id, policy_data, request_id)

        response = {
            'request_id': request_id,
            'entity_id': entity_id,
            'update_status': update_status
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in update_policy_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/receive_state_update', methods=['POST'])
def receive_state_update_endpoint():
    """
    API endpoint for the PE to send state updates to the PA.
    Expects JSON data with 'entity_id', 'new_state', and optional 'request_id'.
    """
    try:
        json_data = request.get_json()
        if not json_data:
            return jsonify({'error': 'No JSON data provided'}), 400

        entity_id = json_data.get('entity_id')
        new_state = json_data.get('new_state')
        request_id = json_data.get('request_id', str(uuid.uuid4()))

        if not entity_id or not new_state:
            return jsonify({'error': 'Missing entity_id or new_state'}), 400

        PA_instance.receive_state_update(entity_id, new_state, request_id)

        response = {
            'request_id': request_id,
            'entity_id': entity_id,
            'status': 'State update processed'
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in receive_state_update_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

# Create an instance of PolicyAdministration
PA_instance = PolicyAdministration()

if __name__ == '__main__':
    # Set the Policy Engine URL before starting the Flask server
    PA_instance.set_pe_url('http://192.52.33.4:5000')  # Ensure the URL includes the protocol (http:// or https://)
    # Run the Flask app on all interfaces, port 5000
    app.run(host='0.0.0.0', port=5000)